# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [12]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [13]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [14]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [15]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [16]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [17]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_assignment 
    WITH REPLICATION=
       { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [18]:
try:
    session.set_keyspace('cassandra_assignment')
except Exception as e:
    print(e)


### Task-1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### For Task-1: A table 'songs_session' with composite primary key sessionId as partition key and itemInSession as clustering key is created to
##### filter the results based on sessionId and itemInSession.

In [19]:
query = "CREATE TABLE IF NOT EXISTS songs_session"
query+= "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, songLength FLOAT, PRIMARY KEY((sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

           

In [20]:
##### Insert values from the csv file into the songs_session table.

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_session (sessionId, itemInSession, artist, song, songLength)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

##### a SELECT to verify that the data have been inserted into the table

In [22]:
query = "SELECT artist, song, songLength FROM songs_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', songlength=495.30731201171875)


### Task-2: Get the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### For Task-2: A table 'song_by_user' with primary keys (userId and sessionId) as partition key and itemInSessionId as clustering key is created to
##### create a query that requires the song to be sorted by itemInSessionId.

In [26]:
query = "CREATE TABLE IF NOT EXISTS song_by_user"
query += "(userId INT, sessionId INT, itemInSessionId INT, artistName TEXT, song TEXT, firstName TEXT, lastName TEXT, PRIMARY KEY((userId, sessionId), itemInSessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [66]:
##### Insert values from the csv file into the song_by_user table.

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user (userId, sessionId, itemInSessionId, artistName, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),  int(line[3]), line[0], line[9], line[1], line[4]))

                    

In [28]:
##### a SELECT to verify that the data have been inserted into the table

In [30]:
query = "SELECT artistName, song, itemInSessionId, firstname, lastname FROM song_by_user WHERE userId = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artistname='Down To The Bone', song="Keep On Keepin' On", iteminsessionid=0, firstname='Sylvie', lastname='Cruz')
Row(artistname='Three Drives', song='Greece 2000', iteminsessionid=1, firstname='Sylvie', lastname='Cruz')
Row(artistname='Sebastien Tellier', song='Kilometer', iteminsessionid=2, firstname='Sylvie', lastname='Cruz')
Row(artistname='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsessionid=3, firstname='Sylvie', lastname='Cruz')


In [31]:
### Task-3: Get the following: user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [32]:
query = "CREATE TABLE IF NOT EXISTS song_by_user_history"
query += "(song TEXT, userId INT, firstname TEXT, lastname TEXT, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [33]:
##### For Task-3: A table 'song_by_user_history' with composite primary keys song as partition key and userId as clustering key is created to
##### create a query that requires the song to be filtered and userId is used to get the unique value.

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user_history (song, userId, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

  

In [ ]:
##### a SELECT to verify that the data have been inserted into the table

In [ ]:
query = "SELECT firstname, lastname FROM song_by_user_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row) 

### Drop the tables before closing out the sessions

In [8]:
query = "drop table songs_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

name 'session' is not defined


In [25]:
query = "drop table song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [10]:
query = "drop table song_by_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

name 'session' is not defined


### Close the session and cluster connection¶

In [11]:
session.shutdown()
cluster.shutdown()

NameError: name 'session' is not defined